In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        self.fc1 = nn.Linear(16 * 6 * 6, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # first conv1 -> relu
        # x = F.relu(self.conv1(x))
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        print("After conv1, shape is".format(x.shape))

        # # second: pooling layer
        # x = F.max_pool2d(x, (2, 2))
        # print("After pooling, shape is".format(x.shape))

        # third: conv2 -> relu
        x = F.relu(self.conv2(x))
        print("After conv2, shape is".format(x.shape))

        # forth: pooling layer
        x = F.max_pool2d(x, (2, 2))
        print("After pooling, shape is".format(x.shape))

        # fifth: fc1
        # flattern
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        print("After fc1, shape is".format(x.shape))

        # sixth: fc2
        x = F.relu(self.fc2(x))
        print("After fc2, shape is".format(x.shape))

        # seventh: fc3
        x = F.relu(self.fc3(x))
        print("After fc3, shape is".format(x.shape))

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s

        return num_features
        

In [4]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [5]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [6]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

After conv1, shape is
After conv2, shape is
After pooling, shape is
After fc1, shape is
After fc2, shape is
After fc3, shape is
tensor([[0.0000, 0.0819, 0.0000, 0.0000, 0.0000, 0.0363, 0.0400, 0.0000, 0.1601,
         0.0000]], grad_fn=<ReluBackward0>)


In [7]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [8]:
input.unsqueeze?

In [9]:
output = net(input)

After conv1, shape is
After conv2, shape is
After pooling, shape is
After fc1, shape is
After fc2, shape is
After fc3, shape is


In [10]:
target = torch.randn(10)  # a dummy target, for example

In [12]:
target.shape

torch.Size([10])

In [13]:
target = target.view(1, -1)

In [14]:
target.shape

torch.Size([1, 10])

In [15]:
criterion = nn.MSELoss()

In [16]:
loss = criterion(output, target)

In [17]:
print(loss)

tensor(1.5210, grad_fn=<MseLossBackward>)


In [18]:
output

tensor([[0.0000, 0.0819, 0.0000, 0.0000, 0.0000, 0.0363, 0.0400, 0.0000, 0.1601,
         0.0000]], grad_fn=<ReluBackward0>)

In [19]:
target

tensor([[ 0.5617,  0.3209, -1.1923, -1.5824, -0.3929,  1.6455, -1.5463, -1.7950,
          0.6234, -1.4884]])

In [20]:
output - target

tensor([[-0.5617, -0.2389,  1.1923,  1.5824,  0.3929, -1.6092,  1.5863,  1.7950,
         -0.4633,  1.4884]], grad_fn=<SubBackward0>)

In [25]:
(output - target).shape

torch.Size([1, 10])

In [27]:
import numpy as np

In [40]:
mse_array = (output-target).detach().numpy() 

In [41]:
mse_array

array([[-0.5617261 , -0.23892716,  1.1922944 ,  1.5824178 ,  0.392925  ,
        -1.6091737 ,  1.5862998 ,  1.794982  , -0.46331987,  1.4883679 ]],
      dtype=float32)

In [45]:
np.sum(mse_array ** 2, axis =1)/10

array([1.5210276], dtype=float32)

In [33]:
np.sum?

In [46]:
print(loss.grad_fn)

In [47]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])


In [48]:
loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad after backward
tensor([ 0.0027,  0.0020, -0.0167, -0.0190,  0.0035, -0.0132])


In [49]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [50]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

After conv1, shape is
After conv2, shape is
After pooling, shape is
After fc1, shape is
After fc2, shape is
After fc3, shape is


In [52]:
print(net.conv1.bias.grad)

tensor([ 0.0009,  0.0008, -0.0183, -0.0215, -0.0016, -0.0136])


In [53]:
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

After conv1, shape is
After conv2, shape is
After pooling, shape is
After fc1, shape is
After fc2, shape is
After fc3, shape is


In [54]:
print(net.conv1.bias.grad)

tensor([-0.0044,  0.0051, -0.0268, -0.0322, -0.0061, -0.0203])
